In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import pudl
import pudl.extract.eia_bulk_elec as ebulk
import pudl.transform.eia_bulk_elec as tbulk

In [ ]:
fpath = Path('/home/tpb/catalyst/workspace/data/eia_bulk_data/ELEC.zip')
assert fpath.exists()

In [ ]:
%%time
raw = ebulk.extract(fpath)

In [ ]:
raw_meta = raw['metadata']
raw_ts = raw['timeseries']

In [ ]:
raw_meta.head(2)

In [ ]:
tbulk._transform_metadata(raw_meta).head(2)

In [ ]:
pd.to_datetime(raw_meta['last_updated'])

In [ ]:
meta = pd.concat([raw_meta.drop(columns=['name']),
                  tbulk._extract_keys_from_name(raw_meta),
                  tbulk._extract_keys_from_series_id(raw_meta)],
                 axis=1
                )

In [ ]:
meta.sample(5)

In [ ]:
meta.shape, raw_ts.shape

In [ ]:
meta.sample(5)

In [ ]:
ts.head(5)

In [ ]:
ts.isna().mean()

In [ ]:
ts.isna().groupby(ts['series_id'].str[-1])['value'].mean()

In [ ]:
ts.memory_usage(deep=True) / 2**20

In [ ]:
meta.describe()

In [ ]:
meta.loc[:, ~meta.isna().all()].describe()

In [ ]:
all_nan = meta.isna().all()
all_none = meta.eq('None').all()
to_drop = all_nan | all_none
dropped_col_names = meta.columns[to_drop]

In [ ]:
set(dropped_col_names)

In [ ]:
tbulk._get_empty_col_names(meta)

In [ ]:
tbulk._get_redundant_frequency_col_names(meta)

In [ ]:
tbulk._get_constant_col_names(meta)

In [ ]:
meta['fuel_code'].value_counts()

In [ ]:
meta.columns[meta.nunique() == 1]

In [ ]:
geo_parts = meta['geoset_id'].str.split('-', expand=True)

In [ ]:
tbulk._get_col_names_to_drop(meta)

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
meta.sample(5, random_state=2)[['series_id', 'description']]

In [ ]:
dmeta = meta.drop(columns=tbulk._get_col_names_to_drop(meta))

In [ ]:
dmeta.sample(5, random_state=3)

In [ ]:
# confirm connecticut is in "geography" because it is missing from "description"
dmeta.loc[dmeta['region_code'].eq("NEW"),:].head(3)

In [ ]:
dmeta.groupby(["fuel", "fuel_code"])['description'].first().str.strip('; ').str.split(';', expand=True)[[0]]

In [ ]:
dmeta.groupby(["sector", "sector_code"])['description'].first().str.strip('; ').str.split(';', expand=True)[[1]]

In [ ]:
from io import StringIO
pd.read_csv(StringIO(
	"""
fuel	fuel_code	description
bituminous coal	BIT	Bituminous Coal
coal	COW	Summation of all types of coal
lignite coal	LIG	Lignite Coal
natural gas	NG	Natural Gas
petroleum coke	PC	Petroleum Coke (solid residual petroluem)
petroleum liquids	PEL	Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids)
subbituminous coal	SUB	Subbituminous Coal
    """
), sep='\t')

In [ ]:
out = StringIO()
dmeta.groupby(["region", "region_code"])['description'].first().str.strip('; ').str.split(';', expand=True)[[2]].to_csv(out, sep=",")

In [ ]:
print(out.getvalue())

In [ ]:
wide_ts = ts.merge(dmeta[['series_id', 'units', 'fuel', 'region','sector', 'frequency']], on='series_id').drop(columns='series_id')

In [ ]:
wide_ts = wide_ts.set_index(['units', 'fuel', 'region','sector', 'frequency', 'date']).unstack('units')

In [ ]:
wide_ts.columns = wide_ts.columns.droplevel(level=None)

In [ ]:
wide_ts.rename(columns={'billion Btu': 'receipts_billion_btu',
                       'dollars per million Btu': 'cost_dollars_per_mmbtu'}, inplace=True)

In [ ]:
wide_ts.sample(5)

In [ ]:
wide_ts.columns.name = None

In [ ]:
wide_ts.columns

In [ ]:
tbulk._transform_timeseries(ts, dmeta).sample(5)

In [ ]:
cen_reg = pd.read_csv(
    StringIO(
        """
region,region_code,included_states
East North Central,ENC,"Illinois,Indiana,Michigan,Ohio,Wisconsin"
East South Central,ESC,"Alabama,Kentucky,Mississippi,Tennessee"
Middle Atlantic,MAT,"New Jersey,New York,Pennsylvania"
Mountain,MTN,"Arizona,Colorado,Idaho,Montana,Nevada,New Mexico,Utah,Wyoming"
New England,NEW,"Maine,Massaschuetts,New Hampshire,Rhode Island,Vermont,Connecticut"
Pacific Contiguous,PCC,"California,Oregon,Washington"
Pacific Noncontiguous,PCN,"Alaska,Hawaii"
South Atlantic,SAT,"Delaware,the District of Columbia,Florida,Georgia,Maryland,North Carolina,South Carolina,Virginia,West Virginia"
West North Central,WNC,"Iowa,Kansas,Minnesota,Missouri,Nebraska,North Dakota,South Dakota"
West South Central,WSC,"Arkansas,Louisiana,Oklahoma,Texas"
    """
    ),
)

In [ ]:
cen_reg

In [ ]:
cen_reg['included_states'].str.split(',', expand=True).melt()

In [ ]:
pd.concat([cen_reg[['region', 'region_code']], cen_reg['included_states'].str.split(',', expand=True)], axis=1).melt(id_vars=['region', 'region_code'], value_name='state').dropna(subset='state').drop(columns='variable').sort_values('region')